### Why pipeline
Pipeline make task efficient

## Roadmap
missing values -- ohe --- scailing ---- feature selection --model(clf)

In [92]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [93]:
df = pd.read_csv(r"C:\Users\Admin\Documents\Jupyter Notebook\Machine Learning (CampusX)\Dataset\train.csv")

In [94]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [95]:
df.drop(['Cabin','Ticket','PassengerId','Name'],axis=1,inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [96]:
X = df.drop(['Survived'],axis=1)
y=df['Survived']

In [97]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

### Note: we dont give name of column in column transformer instead index value bcoz after getting output from first step i.e missing values it will be in numpy array and not dataframe (numpy array has no col name)

In [98]:
## Missing Values imputation
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embark',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [99]:
## OHE
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [100]:
##Scailing  we have 10 columns excluding output col
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,11))  #+1 feature
])

In [101]:
# ## Features Selection
# from sklearn.feature_selection import SelectKBest,chi2
# trf4 = SelectKBest(score_func=chi2,k=8)   ## 8 top columns used

In [102]:
trf5 = DecisionTreeClassifier()

## Pipeline

In [103]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    # ('trf4',trf4),
    ('trf5',trf5)
])

## Pipeline Vs make_pipeline
- Pipeline requires naming of steps,make_pipeline does not require
- (same applies to columnTransformer vs make_column_transformer

In [104]:
# ## Alternate syntax
# pipe = make_pipeline(trf1,trf2,trf3,trf4)

## Train
#### Note : when we do preprocessing using pipeline and no model training then we need to use **.fit_transform function** and when we are including model training then we need to use **.fit**

In [105]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embark',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 11, None))])),
                ('trf5', DecisionTreeClassifier())])

## Explore the pipeline

In [106]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_   #decode

array([29.78590426])

In [107]:
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [108]:
## Predict
y_pred = pipe.predict(X_test)

In [109]:
from sklearn.metrics import accuracy_score
print("Accuracy Score is",accuracy_score(y_test,y_pred))

Accuracy Score is 0.6368715083798883


## Cross Validation using Pipeline
- cv= 5 means 5 times data will be splitted and trained and then taken mean all the output i.e 5
- thats our actual score

In [111]:
## Cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=5,scoring='accuracy').mean()

0.6151876292721363

## GridSearch using Pipeline  (Hyperparameter tuning)

In [113]:
params ={
    'trf5__max_depth': [1,2,3,4,5,None]
}  #GridSearchCV

In [114]:
from sklearn.model_selection import GridSearchCV

In [115]:
grid = GridSearchCV(pipe, params, cv=5, scoring ='accuracy')

In [116]:
grid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embark',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 11, None))])),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [118]:
grid.best_score_

0.6151876292721363

In [119]:
grid.best_params_

{'trf5__max_depth': 1}

## Exporting the pipeline

In [128]:
# # export
# import pickle
# pickle.dump(pipe,open('models/pipe.pkl','wb'))